## Install pyTigerGraph

In [1]:
# Setup
!pip install pyTigerGraph

## Add Imports and Establish Initial Connection

In [2]:
# Imports
import pyTigerGraph as tg
import json
import pandas as pd

# Connection parameters
hostName = "https://language.i.tgcloud.io"
userName = "tigergraph"
password = "tigergraph"

conn = tg.TigerGraphConnection(host=hostName, username=userName, password=password)

print("Connected")

Connected


## Clone the Data

In [3]:
# from google.colab import drive
# drive.mount('/content/drive')

In [4]:
# cd '/content/drive/MyDrive'
root = '/Users/sudha.vijayakumar/Documents/GitHub/LanguageModel/Curated_Data/'

In [5]:
# !git clone https://github.com/sudha-vijayakumar/LanguageModel.git

## Define and Publish the Schema

In [6]:
# DEFINE / CREATE ALL EDGES AND VERTICES 
results = conn.gsql('''
  USE GLOBAL
  CREATE VERTEX words(PRIMARY_ID id STRING, uri STRING, word STRING, pos STRING,definition STRING) WITH STATS="OUTDEGREE_BY_EDGETYPE", PRIMARY_ID_AS_ATTRIBUTE="true"
  CREATE DIRECTED EDGE type(FROM words, TO words, typeOf STRING) WITH REVERSE_EDGE="reverse_type"
  CREATE VERTEX word(PRIMARY_ID id STRING, name STRING, pos STRING) WITH STATS="OUTDEGREE_BY_EDGETYPE", PRIMARY_ID_AS_ATTRIBUTE="true"
  CREATE VERTEX synset(PRIMARY_ID id STRING, definition STRING) WITH STATS="OUTDEGREE_BY_EDGETYPE", PRIMARY_ID_AS_ATTRIBUTE="true"
  CREATE DIRECTED EDGE is_a(FROM synset, TO word|FROM word, TO synset, type STRING) WITH REVERSE_EDGE="reverse_is_a"
''')
print(results)

Successfully created vertex types: [words].
Successfully created edge types: [type].
Successfully created reverse edge types: [reverse_type].
Successfully created vertex types: [word].
Successfully created vertex types: [synset].
Successfully created edge types: [is_a].
Successfully created reverse edge types: [reverse_is_a].


# WordNET

## Create WordNet Graph

In [7]:
results = conn.gsql('CREATE GRAPH WordNet(words,type,reverse_type)')
print(results)

The graph WordNet is created.


In [8]:
conn.graphname="WordNet"
secret = conn.createSecret()
print(secret)
authToken = conn.getToken(secret)
authToken = authToken[0]
print(authToken)
# authToken = 'rc7reopbis1667ksgcppq5v5fb99p6s1'
conn = tg.TigerGraphConnection(host=hostName, graphname="WordNet", username=userName, password=password, apiToken=authToken)

def pprint(string):
  print(json.dumps(string, indent=2))

2saeocjkgqsalca3d2324o3jsa4r7f0s
te75poe9nv7hjsv7autit6ikcs8665jn


## Create Loading Jobs

### Word

Let's take a look at what one of our files looks like so we can write a loading job.

In [9]:
!head -n 2 root+'csv_imports/WN-nodes.csv'

head: root+csv_imports/WN-nodes.csv: No such file or directory


Here it's important to note that the `$0`, `$1` values line up with the columns of your data.
In this example:
- `$0` is the `uri` column,
- `$1` is `id`,
- `$2` is `word`
- and so on

In [10]:
results = conn.gsql('''
  USE GRAPH WordNet
  BEGIN
  CREATE LOADING JOB load_job_WN FOR GRAPH WordNet {
      DEFINE FILENAME MyDataSource;
      LOAD MyDataSource TO EDGE type VALUES($0, $1, $2) USING SEPARATOR=",", HEADER="true", EOL="", QUOTE="double";
    }
  END
  ''')
print(results)

Using graph 'WordNet'
Successfully created loading jobs: [load_job_WN].


### Word Edges

In [11]:
results = conn.gsql('''
  USE GRAPH WordNet
  BEGIN
  CREATE LOADING JOB load_job_WN_nodes FOR GRAPH WordNet {
      DEFINE FILENAME MyDataSource;
      LOAD MyDataSource TO VERTEX words VALUES($1, $0, $2, $3, $4) USING SEPARATOR=",", HEADER="true", EOL="", QUOTE="double";
    }
  END
  ''')
print(results)

Using graph 'WordNet'
Successfully created loading jobs: [load_job_WN_nodes].


## Load Data

### Words

In [12]:
# Load the posts file wiht the 'load_words' job
load_words = root+'csv_imports/WN-nodes.csv'
results = conn.uploadFile(load_words, fileTag='MyDataSource', jobName='load_job_WN_nodes')
print(json.dumps(results, indent=2))

[
  {
    "sourceFileName": "Online_POST",
    "statistics": {
      "validLine": 117792,
      "rejectLine": 0,
      "failedConditionLine": 0,
      "notEnoughToken": 0,
      "invalidJson": 0,
      "oversizeToken": 0,
      "vertex": [
        {
          "typeName": "words",
          "validObject": 117792,
          "noIdFound": 0,
          "invalidAttribute": 0,
          "invalidVertexType": 0,
          "invalidPrimaryId": 0,
          "invalidSecondaryId": 0,
          "incorrectFixedBinaryLength": 0
        }
      ],
      "edge": [],
      "deleteVertex": [],
      "deleteEdge": []
    }
  }
]


### Edges

In [13]:
# Load the posts file wiht the 'load_edges' job
load_edges = root+'csv_imports/WN-edges.csv'
results = conn.uploadFile(load_edges, fileTag='MyDataSource', jobName='load_job_WN')
print(json.dumps(results, indent=2))

[
  {
    "sourceFileName": "Online_POST",
    "statistics": {
      "validLine": 293652,
      "rejectLine": 0,
      "failedConditionLine": 0,
      "notEnoughToken": 0,
      "invalidJson": 0,
      "oversizeToken": 0,
      "vertex": [],
      "edge": [
        {
          "typeName": "type",
          "validObject": 293652,
          "noIdFound": 0,
          "invalidAttribute": 0,
          "invalidVertexType": 0,
          "invalidPrimaryId": 0,
          "invalidSecondaryId": 0,
          "incorrectFixedBinaryLength": 0
        }
      ],
      "deleteVertex": [],
      "deleteEdge": []
    }
  }
]


## Exploring the Graph

### Get Vertex and Edge Schema

In [14]:
results = conn.getVertexTypes()
print(f"Verticies: {results}")
vertices = results

results = conn.getEdgeTypes()
print(f"Edges: {results}")
edges = results

Verticies: ['words']
Edges: ['type']


In [15]:

print(f"Results for Post vertex")
pprint(conn.getVertexType("words"))

print("-----------------")
print(f"Results for liked edge")
pprint(conn.getEdgeType("type"))


Results for Post vertex
{
  "Config": {
    "TAGGABLE": false,
    "STATS": "OUTDEGREE_BY_EDGETYPE",
    "PRIMARY_ID_AS_ATTRIBUTE": true
  },
  "Attributes": [
    {
      "AttributeType": {
        "Name": "STRING"
      },
      "IsPartOfCompositeKey": false,
      "PrimaryIdAsAttribute": false,
      "AttributeName": "uri",
      "HasIndex": false,
      "internalAttribute": false,
      "IsPrimaryKey": false
    },
    {
      "AttributeType": {
        "Name": "STRING"
      },
      "IsPartOfCompositeKey": false,
      "PrimaryIdAsAttribute": false,
      "AttributeName": "word",
      "HasIndex": false,
      "internalAttribute": false,
      "IsPrimaryKey": false
    },
    {
      "AttributeType": {
        "Name": "STRING"
      },
      "IsPartOfCompositeKey": false,
      "PrimaryIdAsAttribute": false,
      "AttributeName": "pos",
      "HasIndex": false,
      "internalAttribute": false,
      "IsPrimaryKey": false
    },
    {
      "AttributeType": {
        "Name": "ST

## Counting Data

In [16]:
print("Vertex Counts")
for vertex in vertices:
  print(f"There are {conn.getVertexCount(vertex)} {vertex} vertices in the graph")

print("--------------")
print("Edge Counts")
for edge in edges:
  print(f"There are {conn.getEdgeCount(edge)} {edge} edges in the graph")

Vertex Counts
There are 33626 words vertices in the graph
--------------
Edge Counts
There are 0 type edges in the graph


## Extracting Data

### Vertex/Edge Set Format

#### Getting a Vertex

In [17]:
results = conn.getVerticesById("words", "15314760-n")
pprint(results)

[
  {
    "v_id": "15314760-n",
    "v_type": "words",
    "attributes": {
      "id": "15314760-n",
      "uri": "http://wordnet-rdf.princeton.edu/id/15314760-n",
      "word": "lead_time",
      "pos": "noun",
      "definition": "the time interval between the initiation and the completion of a production process; &quot;the lead times for many publications can vary tremendously&quot;; &quot;planning is an area where lead time can be reduced&quot;@en"
    }
  }
]


#### Or Multiple Vertices

In [18]:
tdf1 = conn.getVerticesById("words", ["15137796-n","15192825-n"])
pprint(tdf1)

[
  {
    "v_id": "15137796-n",
    "v_type": "words",
    "attributes": {
      "id": "15137796-n",
      "uri": "http://wordnet-rdf.princeton.edu/id/15137796-n",
      "word": "period",
      "pos": "noun",
      "definition": "an amount of time; &quot;a time period of 30 years&quot;; &quot;hastened the period of time of his recovery&quot;; &quot;Picasso's blue period&quot;@en"
    }
  },
  {
    "v_id": "15192825-n",
    "v_type": "words",
    "attributes": {
      "id": "15192825-n",
      "uri": "http://wordnet-rdf.princeton.edu/id/15192825-n",
      "word": "eve",
      "pos": "noun",
      "definition": "the period immediately before something; &quot;on the eve of the French Revolution&quot;@en"
    }
  }
]


#### Count Edges Connected to a Vertex

In [19]:
results = conn.getEdgeCountFrom("words", "15192825-n")
pprint(results)

{
  "type": 1,
  "reverse_type": 1
}


#### Show all Edges Connected to a Vertex

In [20]:
results = conn.getEdges("words", "15192825-n")
pprint(results)

[
  {
    "e_type": "type",
    "directed": true,
    "from_id": "15192825-n",
    "from_type": "words",
    "to_id": "15137796-n",
    "to_type": "words",
    "attributes": {
      "typeOf": "hypernym"
    }
  },
  {
    "e_type": "reverse_type",
    "directed": true,
    "from_id": "15192825-n",
    "from_type": "words",
    "to_id": "15137796-n",
    "to_type": "words",
    "attributes": {
      "typeOf": "hyponym"
    }
  }
]


### As Pandas Dataframe
Supports all of the above in native Pandas Dataframe format.

#### All Vertices of one Type

In [21]:
df1 = conn.getVertexDataframe("words")
print(df1)

              v_id          id  \
0       15310990-n  15310990-n   
1       15309261-n  15309261-n   
2       15268162-n  15268162-n   
3       15267410-n  15267410-n   
4       15247336-n  15247336-n   
...            ...         ...   
117789  06698720-n  06698720-n   
117790  04766683-n  04766683-n   
117791  04767551-n  04767551-n   
117792  01655911-a  01655911-a   
117793  04786292-n  04786292-n   

                                                   uri                word  \
0       http://wordnet-rdf.princeton.edu/id/15310990-n  neonatal_mortality   
1       http://wordnet-rdf.princeton.edu/id/15309261-n   peculiar_velocity   
2       http://wordnet-rdf.princeton.edu/id/15268162-n           long_haul   
3       http://wordnet-rdf.princeton.edu/id/15267410-n        Whit-Tuesday   
4       http://wordnet-rdf.princeton.edu/id/15247336-n          school_day   
...                                                ...                 ...   
117789  http://wordnet-rdf.princeton.edu/id/0

#### One or More Vertex

In [22]:
df2 = conn.getVertexDataframeById("words", ["15192825-n"])
print(df2)

         v_id          id                                             uri  \
0  15192825-n  15192825-n  http://wordnet-rdf.princeton.edu/id/15192825-n   

  word   pos                                         definition  
0  eve  noun  the period immediately before something; &quot...  


#### Convert Vertex/Edge Set to Dataframe
We'll use the results from the 'Or Multiple Vertices' cell. 

In [23]:
df3 = conn.vertexSetToDataFrame(tdf1)
print(df3)

         v_id          id                                             uri  \
0  15137796-n  15137796-n  http://wordnet-rdf.princeton.edu/id/15137796-n   
1  15192825-n  15192825-n  http://wordnet-rdf.princeton.edu/id/15192825-n   

     word   pos                                         definition  
0  period  noun  an amount of time; &quot;a time period of 30 y...  
1     eve  noun  the period immediately before something; &quot...  


#### Get Edges

In [24]:
df4 = conn.getEdgesDataframe("words", "15192825-n", limit=3)
print(df4)

  from_type     from_id to_type       to_id    typeOf
0     words  15192825-n   words  15137796-n  hypernym
1     words  15192825-n   words  15137796-n   hyponym


## Path Finding
Find paths between vertices.

Supported are:
- shortestPath - one shortest path between vertices
- allPaths - all paths within the specified edge limit

In [25]:
results = conn.shortestPath([("words", "15192825-n")], [("words", "15161294-n")])
pprint(results)

[{'type': 'words', 'id': '15192825-n'}]
[{'type': 'words', 'id': '15161294-n'}]
[
  {
    "vertices": [
      {
        "v_id": "15137796-n",
        "v_type": "words",
        "attributes": {
          "id": "15137796-n",
          "uri": "http://wordnet-rdf.princeton.edu/id/15137796-n",
          "word": "period",
          "pos": "noun",
          "definition": "an amount of time; &quot;a time period of 30 years&quot;; &quot;hastened the period of time of his recovery&quot;; &quot;Picasso's blue period&quot;@en"
        }
      },
      {
        "v_id": "15161294-n",
        "v_type": "words",
        "attributes": {
          "id": "15161294-n",
          "uri": "http://wordnet-rdf.princeton.edu/id/15161294-n",
          "word": "midweek",
          "pos": "noun",
          "definition": "the middle of a week@en"
        }
      },
      {
        "v_id": "15192825-n",
        "v_type": "words",
        "attributes": {
          "id": "15192825-n",
          "uri": "http://wordnet

# ConceptNET

## Create WordNet Graph

In [26]:
results = conn.gsql('CREATE GRAPH ConceptNET(word, synset, is_a, reverse_is_a)')
print(results)

The graph ConceptNET is created.


In [27]:
conn.graphname="ConceptNET"
secret = conn.createSecret()
print(secret)
authToken = conn.getToken(secret)
authToken = authToken[0]
print(authToken)
conn = tg.TigerGraphConnection(host=hostName, graphname="ConceptNET", username=userName, password=password, apiToken=authToken)

def pprint(string):
  print(json.dumps(string, indent=2))

aapqehnk14btdnacrsvsmglrdm746snq
mul891oht9idb2ocmacnh59nfi5l1me0


## Create Loading Jobs

### Concept/ Words Edges

Let's take a look at what one of our files looks like so we can write a loading job. 

- has_synset(FROM rootWord, TO synset)
is_a_synset_of(FROM synset, TO rootWord)


In [28]:
!head -n 2 '/content/drive/MyDrive/LanguageModel/Data_Processing/csv_imports/words.csv'

head: /content/drive/MyDrive/LanguageModel/Data_Processing/csv_imports/words.csv: No such file or directory


Here it's important to note that the `$0`, `$1` values line up with the columns of your data.
In this example:
- `$0` is the `uri` column,
- `$1` is `id`,
- `$2` is `word`
- and so on

In [29]:
results = conn.gsql('''
  USE GRAPH ConceptNET
  BEGIN
  CREATE LOADING JOB load_job_words FOR GRAPH ConceptNET {
      DEFINE FILENAME MyDataSource;
      LOAD MyDataSource TO VERTEX word VALUES($0, $1, $2) USING SEPARATOR=",", HEADER="true", EOL="", QUOTE="double";
    }

    CREATE LOADING JOB load_job_synsets FOR GRAPH ConceptNET {
      DEFINE FILENAME MyDataSource;
      LOAD MyDataSource TO VERTEX synset VALUES($0, $2) USING SEPARATOR=",", HEADER="true", EOL="", QUOTE="double";
    }


    CREATE LOADING JOB load_job_relationships FOR GRAPH ConceptNET {
      DEFINE FILENAME MyDataSource;
      LOAD MyDataSource TO EDGE is_a VALUES($0 word, $1 synset, $4) USING SEPARATOR=",", HEADER="true", EOL="";
      LOAD MyDataSource TO EDGE is_a VALUES($0 synset, $1 word, $4) USING SEPARATOR=",", HEADER="true", EOL="";
    }

  END
  ''')
print(results)

Using graph 'ConceptNET'
Successfully created loading jobs: [load_job_words].
Successfully created loading jobs: [load_job_synsets].
Successfully created loading jobs: [load_job_relationships].


### Concept/ Word Root


In [30]:
!head -n 2 '/content/drive/MyDrive/LanguageModel/Data_Processing/csv_imports/synsets.csv'

head: /content/drive/MyDrive/LanguageModel/Data_Processing/csv_imports/synsets.csv: No such file or directory


In [31]:
results = conn.gsql('''
  USE GRAPH ConceptNET
  BEGIN
  CREATE LOADING JOB load_job_words_csv_1644515929297 FOR GRAPH ConceptNET {
      DEFINE FILENAME MyDataSource;
      LOAD MyDataSource TO VERTEX word VALUES($0, $1, $2) USING SEPARATOR=",", HEADER="true", EOL="\n", QUOTE="double";
    }

  CREATE LOADING JOB load_job_relationships_csv_1644515956497 FOR GRAPH ConceptNET {
      DEFINE FILENAME MyDataSource;
      LOAD MyDataSource TO EDGE is_a VALUES($0 synset, $1 word, $4) USING SEPARATOR=",", HEADER="true", EOL="\n", QUOTE="double";
      LOAD MyDataSource TO EDGE is_a VALUES($0 word, $1 synset, $4) USING SEPARATOR=",", HEADER="true", EOL="\n", QUOTE="double";
    }

  CREATE LOADING JOB load_job_synsets_csv_1644515968121 FOR GRAPH ConceptNET {
      DEFINE FILENAME MyDataSource;
      LOAD MyDataSource TO VERTEX synset VALUES($0, $2) USING SEPARATOR=",", HEADER="true", EOL="\n", QUOTE="double";
    }
  END
  ''')
print(results)

Using graph 'ConceptNET'
Successfully created loading jobs: [load_job_words_csv_1644515929297].
Successfully created loading jobs: [load_job_relationships_csv_1644515956497].
Successfully created loading jobs: [load_job_synsets_csv_1644515968121].


### Concept/ Synset 
- synset(PRIMARY_ID id STRING, pos STRING, definition STRING, label STRING) WITH STATS="OUTDEGREE_BY_EDGETYPE", PRIMARY_ID_AS_ATTRIBUTE="true"


In [32]:
results = conn.gsql('''
  USE GRAPH ConceptNET
  BEGIN
  CREATE LOADING JOB load_job_relationships FOR GRAPH ConceptNET {
      DEFINE FILENAME MyDataSource;
      LOAD MyDataSource TO EDGE is_a VALUES($0 word, $1 synset, $4) USING SEPARATOR=",", HEADER="true", EOL="\n";
      LOAD MyDataSource TO EDGE is_a VALUES($0 synset, $1 word, $4) USING SEPARATOR=",", HEADER="true", EOL="\n";
    }
  END
  ''')
print(results)

Using graph 'ConceptNET'
Semantic Check Fails: The job name load_job_relationships already exists in other objects!
Failed to create loading jobs: [load_job_relationships].


## Load Data

### Words

In [41]:
# Load the posts file wiht the 'load_words' job
load_words = root+'csv_imports/words.csv'
results = conn.uploadFile(load_words, timeout='100000',fileTag='MyDataSource', jobName='load_job_words_csv_1644515929297')
print(json.dumps(results, indent=2))

[
  {
    "sourceFileName": "Online_POST",
    "statistics": {
      "validLine": 1530137,
      "rejectLine": 0,
      "failedConditionLine": 0,
      "notEnoughToken": 0,
      "invalidJson": 0,
      "oversizeToken": 0,
      "vertex": [
        {
          "typeName": "word",
          "validObject": 1530137,
          "noIdFound": 0,
          "invalidAttribute": 0,
          "invalidVertexType": 0,
          "invalidPrimaryId": 0,
          "invalidSecondaryId": 0,
          "incorrectFixedBinaryLength": 0
        }
      ],
      "edge": [],
      "deleteVertex": [],
      "deleteEdge": []
    }
  }
]


### Synsets

In [42]:
# Load the posts file wiht the 'load_synsets' job
load_synsets = root+'csv_imports/synsets.csv'
results = conn.uploadFile(load_synsets, timeout='100000', fileTag='MyDataSource', jobName='load_job_synsets_csv_1644515968121')
print(json.dumps(results, indent=2))

[
  {
    "sourceFileName": "Online_POST",
    "statistics": {
      "validLine": 117660,
      "rejectLine": 0,
      "failedConditionLine": 0,
      "notEnoughToken": 0,
      "invalidJson": 0,
      "oversizeToken": 0,
      "vertex": [
        {
          "typeName": "synset",
          "validObject": 117660,
          "noIdFound": 0,
          "invalidAttribute": 0,
          "invalidVertexType": 0,
          "invalidPrimaryId": 0,
          "invalidSecondaryId": 0,
          "incorrectFixedBinaryLength": 0
        }
      ],
      "edge": [],
      "deleteVertex": [],
      "deleteEdge": []
    }
  }
]


### Edges

In [43]:
# Load the posts file wiht the 'load_edges' job
load_edges = root+'relationships.csv'
results = conn.uploadFile(load_edges, timeout='100000', fileTag='MyDataSource', jobName='load_job_relationships_csv_1644515956497')
print(json.dumps(results, indent=2))

null


## Exploring the Graph

### Get Vertex and Edge Schema

In [44]:
results = conn.getVertexTypes()
print(f"Verticies: {results}")
vertices = results

results = conn.getEdgeTypes()
print(f"Edges: {results}")
edges = results

Verticies: ['word', 'synset']
Edges: ['is_a']


In [45]:

# print(f"Results for Post vertex")
# pprint(conn.getVertexType("word"))

print("-----------------")
print(f"Results for liked edge")
pprint(conn.getEdgeType("is_a"))


-----------------
Results for liked edge
{
  "IsDirected": true,
  "ToVertexTypeName": "*",
  "Config": {
    "REVERSE_EDGE": "reverse_is_a"
  },
  "Attributes": [
    {
      "AttributeType": {
        "Name": "STRING"
      },
      "IsPartOfCompositeKey": false,
      "PrimaryIdAsAttribute": false,
      "AttributeName": "type",
      "HasIndex": false,
      "internalAttribute": false,
      "IsPrimaryKey": false
    }
  ],
  "FromVertexTypeName": "*",
  "EdgePairs": [
    {
      "From": "synset",
      "To": "word"
    },
    {
      "From": "word",
      "To": "synset"
    }
  ],
  "Name": "is_a"
}


## Counting Data

In [46]:
print("Vertex Counts")
for vertex in vertices:
  print(f"There are {conn.getVertexCount(vertex)} {vertex} vertices in the graph")

print("--------------")
print("Edge Counts")
for edge in edges:
  print(f"There are {conn.getEdgeCount(edge)} {edge} edges in the graph")

Vertex Counts
There are 334728 word vertices in the graph
There are 0 synset vertices in the graph
--------------
Edge Counts
There are 0 is_a edges in the graph


## Extracting Data

### Vertex/Edge Set Format

#### Getting a Vertex

In [47]:
results = conn.getVerticesById("synset", "judicially.r.01")
pprint(results)

[
  {
    "v_id": "judicially.r.01",
    "v_type": "synset",
    "attributes": {
      "id": "judicially.r.01",
      "definition": "as ordered by a court"
    }
  }
]


#### Or Multiple Vertices

In [52]:
tdf1 = conn.getVerticesById("synset", ["judicially.r.01","clash.v.02"])
pprint(tdf1)

[
  {
    "v_id": "judicially.r.01",
    "v_type": "synset",
    "attributes": {
      "id": "judicially.r.01",
      "definition": "as ordered by a court"
    }
  },
  {
    "v_id": "clash.v.02",
    "v_type": "synset",
    "attributes": {
      "id": "clash.v.02",
      "definition": "be incompatible; be or come into conflict"
    }
  }
]


#### Count Edges Connected to a Vertex

In [ ]:
results = conn.getEdgeCountFrom("word", "conflict.v")
pprint(results)

{
  "is_a": 0,
  "reverse_is_a": 0
}


#### Show all Edges Connected to a Vertex

In [ ]:
results = conn.getEdges("word", "conflict.v")
pprint(results)

[]


### As Pandas Dataframe
Supports all of the above in native Pandas Dataframe format.

#### All Vertices of one Type

In [ ]:
df1 = conn.getVertexDataframe("word")
print(df1)

#### One or More Vertex

In [ ]:
df2 = conn.getVertexDataframeById("word", "conflict.v")
print(df2)

#### Convert Vertex/Edge Set to Dataframe
We'll use the results from the 'Or Multiple Vertices' cell. 

In [ ]:
df3 = conn.vertexSetToDataFrame(tdf1)
print(df3)

#### Get Edges

In [ ]:
df4 = conn.getEdgesDataframe("word", "conflict.v",limit=3)
print(df4)

## Path Finding
Find paths between vertices.

Supported are:
- shortestPath - one shortest path between vertices
- allPaths - all paths within the specified edge limit

In [ ]:
results = conn.shortestPath([("word", "in_due_time.r")], [("word", "in_due_season.r")])
pprint(results)